## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/mlfoundry-examples/blob/main/examples/sklearn/iris_classification.ipynb)

## Install dependencies

In [ ]:
! pip install --quiet "numpy>=1.0.0,<2.0.0" "pandas>=1.0.0,<2.0.0" "matplotlib>=3.5.2,<3.6.0" scikit-learn shap==0.40.0
! pip install -U mlfoundry

## Initialize MLFoundry Client

In [ ]:
import os
import getpass
import urllib.parse
import mlfoundry as mlf

In [ ]:
TFY_URL = os.environ.get('TFY_URL', 'https://app.truefoundry.com/')
TFY_API_KEY = os.environ.get('TFY_API_KEY')
if not TFY_API_KEY:
    print(f'Paste your TrueFoundry API key\nYou can find it over at {urllib.parse.urljoin(TFY_URL, "settings")}')
    TFY_API_KEY = getpass.getpass()

In [ ]:
client = mlf.get_client(api_key=TFY_API_KEY)

---

## Iris plants detection as a Classification problem

In [ ]:
import shap
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

import mlfoundry as mlf

### Loading data and preprocessing

In [ ]:
data = datasets.load_iris()
print(data.keys())

In [ ]:
print(data.DESCR) 

In [ ]:
# Read the DataFrame, first using the feature data
df = pd.DataFrame(data.data, columns=data.feature_names)
# Add a target column, and fill it with the target data
df['target'] = data.target
# Show the first five rows
df.head()

### Split Dataset into Training and Validation

In [ ]:
# Store the feature data
X = pd.DataFrame(data.data, columns=data.feature_names)
# store the target data
y = data.target

In [ ]:
# split the data using scikit-learn's train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y, random_state=42)
print('Train samples:', len(X_train))
print('Test samples:', len(X_test))

### Start MLFoundry Run

In [ ]:
run = client.create_run(project_name='iris-project')
print('RUN 1 ID:', run.run_id)
print(f'You can track your runs live at {urllib.parse.urljoin(TFY_URL, "mlfoundry")}')

### Setting tags

In [ ]:
clf = SVC(gamma='scale', kernel='rbf', probability=True)
run.set_tags({'framework': 'sklearn', 'task': 'classification'})

### Training model, logging parameters and logging model

In [ ]:
clf.fit(X_train, y_train)
print(clf.get_params())
run.log_params(clf.get_params())
run.log_params({'classes': clf.classes_, 'features': clf.n_features_in_})
run.log_model(clf, framework=mlf.ModelFramework.SKLEARN)

### Computing predictions

In [ ]:
# logging predictions
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

### Logging metrics

In [ ]:
metrics = {
    'train/accuracy_score': accuracy_score(y_train, y_pred_train),
    'train/f1': f1_score(y_train, y_pred_train, average='weighted'),
    'test/accuracy_score': accuracy_score(y_test, y_pred_test),
    'test/f1': f1_score(y_test, y_pred_test, average='weighted'),
}
print('Tree 1 metrics:', metrics)
run.log_metrics(metrics)

### Log the dataset

In [ ]:
run.log_dataset(
    dataset_name='train',
    features=X_train,
    predictions=y_pred_train,
    actuals=y_train,
)

In [ ]:
run.log_dataset(
    dataset_name='test',
    features=X_test,
    predictions=y_pred_test,
    actuals=y_test,
)

### Log dataset stats

In [ ]:
explainer = shap.KernelExplainer(model=clf.predict_proba, data=X_train.copy(), link='logit')
shap_values = explainer.shap_values(X_train)
shap.initjs()
shap.summary_plot(shap_values, X_train)

X_train_df = X_train.copy()
X_train_df['targets'] = list(y_train)
X_train_df['predictions'] = list(y_pred_train)
X_train_df['prediction_probabilities'] = list(clf.predict_proba(X_train))


run.log_dataset_stats(
    X_train_df, 
    data_slice='train',
    data_schema=mlf.Schema(
        feature_column_names=list(data.feature_names),
        prediction_column_name='predictions',
        actual_column_name='targets',
        prediction_probability_column_name='prediction_probabilities'
    ),
    shap_values=shap_values,
    model_type='multiclass_classification',
)

In [ ]:
explainer = shap.KernelExplainer(model=clf.predict_proba, data=X_train.copy(), link='logit')
shap_values = explainer.shap_values(X_test)
shap.initjs()
shap.summary_plot(shap_values, X_test)

X_test_df = X_test.copy()
X_test_df['targets'] = list(y_test)
X_test_df['predictions'] = list(y_pred_test)
X_test_df['prediction_probabilities'] = list(clf.predict_proba(X_test))

run.log_dataset_stats(
    X_test_df, 
    data_slice='test',
    data_schema=mlf.Schema(
        feature_column_names=list(data.feature_names),
        prediction_column_name='predictions',
        actual_column_name='targets',
        prediction_probability_column_name='prediction_probabilities'
    ),
    shap_values=shap_values,
    model_type='multiclass_classification',
)

In [ ]:
run.end()